# Technical challenge - Cristiano Martins Monteiro

Importing the libraries

In [93]:
import pandas as pd
import numpy as np
from pulp import LpProblem, LpVariable, LpBinary, get_solver, PULP_CBC_CMD

Defining the classes

In [94]:
# This Item class is a super-class for Cylinder, Box and Container. It defines the instances' identifier and optimization variables.
class Item:
    def __init__(self, identifier):
        self.identifier = identifier

        # In this proposed formulation, the boxes do not need a variable.
        if not isinstance(self, Box):
            self.variable = LpVariable(name=identifier, cat=LpBinary)

    # Converts a list of Item's variables to a Numpy array and sums it.
    @staticmethod
    def sum_variables(list_of_items):
        numpy_array = np.asarray([item.variable for item in list_of_items])
        return np.sum(numpy_array)
    
    # Builds a Numpy array of cylinder variables multiplying each cylinder's weight and volume.
    @staticmethod
    def volume_weight_times_cylinders_vars(list_of_cylinders):
        # Builds an array with two columns: the first one multiplying the volumes to the variables,
        #                                   and the second one multiplying the weights to the variables.
        numpy_array = np.asarray([( cylinder.volume * cylinder.variable,
                                    cylinder.weight * cylinder.variable) for cylinder in list_of_cylinders])

        # Sum the arrays according with the columns (axis = 1). This sum will return one value for each column (volume and weight).
        result = np.sum(numpy_array, axis=1)
        return result[0], result[1]

# Defines the cylinders.
class Cylinder(Item):
    def __init__(self, identifier, weight, volume, density, box):
        super().__init__(identifier)

        self.weight = weight
        self.volume = volume
        self.density = density

        # Each cylinder is inside a box. Both cylinder and box has a reference to each other.
        self.box = box
        self.box.cylinders.append(self)

# Defines the boxes.
class Box(Item):
    def __init__(self, identifier, container):
        super().__init__(identifier)

        # Maintains the box's cylinders.
        self.cylinders = list()

        # Each box is inside a container. Both box and container has a reference to each other.
        self.container = container
        self.container.boxes.append(self)

# Defines the containers.
class Container(Item):
    def __init__(self, identifier):
        super().__init__(identifier)

        # Maintains the container's boxes.
        self.boxes = list()

Reading the data and constructing the instances

In [95]:
# Loading a data frame with only the columns and rows with data.
data_frame = pd.read_excel('Avaliacao_Otimizacao.xlsx', usecols='C:H', skiprows=18)

# Defining dictionaries to store all items. The key for each item is its identifier attribute.
containers = dict()
boxes = dict()
cylinders = dict()
# Iterating over the data
for index, row in data_frame.iterrows():
    # Setting the identifier of each item
    container_id = row['Container']
    box_id = container_id + '-' + row['Box']
    cylinder_id = box_id + '-' + str(row['Cylinder'])

    # Checking if it is the first time this container appears. If not, this container is constructed and stored in the dictionary.
    if container_id not in containers:
        container = Container(identifier=container_id)
        containers[container_id] = container
    # Accessing the already constructed container.
    else:
        container = containers[container_id]
    
    # Checking and doing the same, but for the box.
    if box_id not in boxes:
        box = Box(identifier=box_id, container=container)
        boxes[box_id] = box
    else:
        box = boxes[box_id]
    
    # Checking is not required for cylinders because every line in the data frame is a new cylinder.
    cylinder = Cylinder(identifier=cylinder_id,
                        weight=row['Cylinder weight (g)'],
                        volume=row['Cylinder volume (mL)'],
                        density=row['Density (g/mL)'],
                        box=box)
    cylinders[cylinder_id] = cylinder

Building the optimization model

In [96]:
# Defining the optimization model object.
problem = LpProblem(name="Raizen_Challenge")

# Defining the objective function. For this problem, any feasible solution is good enough.
sum_all_containers_vars = Item.sum_variables(containers.values())
sum_all_cylinders_vars = Item.sum_variables(cylinders.values())
problem += 0 * sum_all_containers_vars + 0 * sum_all_cylinders_vars, "Sum of Zeros. Any feasible solution is good enough."

# Defining the constraints. The constraints are defined in the same order they are described in the challenge.
problem += sum_all_containers_vars == 35, "Choosing exactly 35 containers."

sum_volume_cylinders_vars, sum_weight_cylinders_vars = Item.volume_weight_times_cylinders_vars(cylinders.values())
TOTAL_VOLUME = 5163.69
problem += sum_volume_cylinders_vars == TOTAL_VOLUME, "The selected cylinders' volume must sum to this value."

TOTAL_WEIGHT = 18844
problem += sum_weight_cylinders_vars == TOTAL_WEIGHT, "The selected cylinders together must weigh this much."

for container in containers.values():
    for box in container.boxes:
        sum_cylinders_of_box_vars = Item.sum_variables(box.cylinders)
        # If a container is selected, at least one cylinder from any of its boxes must be selected as well.
        # The len(box.cylinders) works as a Big M.
        problem += len(box.cylinders) * container.variable >= sum_cylinders_of_box_vars, "At least one box must be used for each chosen container " + box.identifier + "."
        problem += sum_cylinders_of_box_vars <= 1, "Choosing at maximum one cylinder per box " + box.identifier + "."

Solving the MIP

In [97]:
solver = get_solver('COIN_CMD')
problem.solve(solver)

Welcome to the CBC MILP Solver 
Version: 2.10.5 
Build Date: Oct 15 2020 

command line - cbc /tmp/0f76263493f649638a0a32c2b78636e1-pulp.mps timeMode elapsed branch printingOptions all solution /tmp/0f76263493f649638a0a32c2b78636e1-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 126 COLUMNS
At line 1953 RHS
At line 2075 BOUNDS
At line 2528 ENDATA
Problem MODEL has 121 rows, 452 columns and 923 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Problem is infeasible - 0.00 seconds
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.01



-1